In [128]:
import numpy as np
import pandas as pd
import datetime

In [129]:
df = pd.read_csv("archives/travel-times.csv")
df.head()

,Date,StartTime,DayOfWeek,GoingTo,Distance,MaxSpeed,AvgSpeed,AvgMovingSpeed,FuelEconomy,TotalTime,MovingTime,Take407All,Comments
0,1/6/2012,16:37,Friday,Home,51.29,127.4,78.3,84.8,NaN,39.3,36.3,No,NaN
1,1/6/2012,08:20,Friday,GSK,51.63,130.3,81.8,88.9,NaN,37.9,34.9,No,NaN
2,1/4/2012,16:17,Wednesday,Home,51.27,127.4,82.0,85.8,NaN,37.5,35.9,No,NaN
3,1/4/2012,07:53,Wednesday,GSK,49.17,132.3,74.2,82.9,NaN,39.8,35.6,No,NaN
4,1/3/2012,18:57,Tuesday,Home,51.15,136.2,83.4,88.1,NaN,36.8,34.8,No,NaN


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            205 non-null    object 
 1   StartTime       205 non-null    object 
 2   DayOfWeek       205 non-null    object 
 3   GoingTo         205 non-null    object 
 4   Distance        205 non-null    float64
 5   MaxSpeed        205 non-null    float64
 6   AvgSpeed        205 non-null    float64
 7   AvgMovingSpeed  205 non-null    float64
 8   FuelEconomy     188 non-null    object 
 9   TotalTime       205 non-null    float64
 10  MovingTime      205 non-null    float64
 11  Take407All      205 non-null    object 
 12  Comments        24 non-null     object 
dtypes: float64(6), object(7)
memory usage: 20.9+ KB


In [131]:
df.isna().sum()

Date                0
StartTime           0
DayOfWeek           0
GoingTo             0
Distance            0
MaxSpeed            0
AvgSpeed            0
AvgMovingSpeed      0
FuelEconomy        17
TotalTime           0
MovingTime          0
Take407All          0
Comments          181
dtype: int64

## Calcular o endtime (StartTime + TotalTime)

In [132]:
df["EndTime"] = pd.to_datetime(df["Date"] + " " + df["StartTime"])
df["EndTime"] = df["EndTime"] + pd.to_timedelta(df["TotalTime"], unit="m")

In [133]:
df.head()

,Date,StartTime,DayOfWeek,GoingTo,Distance,MaxSpeed,AvgSpeed,AvgMovingSpeed,FuelEconomy,TotalTime,MovingTime,Take407All,Comments,EndTime
0,1/6/2012,16:37,Friday,Home,51.29,127.4,78.3,84.8,NaN,39.3,36.3,No,NaN,2012-01-06 17:16:18
1,1/6/2012,08:20,Friday,GSK,51.63,130.3,81.8,88.9,NaN,37.9,34.9,No,NaN,2012-01-06 08:57:54
2,1/4/2012,16:17,Wednesday,Home,51.27,127.4,82.0,85.8,NaN,37.5,35.9,No,NaN,2012-01-04 16:54:30
3,1/4/2012,07:53,Wednesday,GSK,49.17,132.3,74.2,82.9,NaN,39.8,35.6,No,NaN,2012-01-04 08:32:48
4,1/3/2012,18:57,Tuesday,Home,51.15,136.2,83.4,88.1,NaN,36.8,34.8,No,NaN,2012-01-03 19:33:48


## Agrupa por día da semana e indica a media que lle leva chegar (TotalTime) por día da semana

In [134]:
# df["Date"] = df["Date"].astype("datetime64[ns]")
df["Date"] = pd.to_datetime(df["Date"])

In [135]:
df.groupby(df["DayOfWeek"])["TotalTime"].mean()

DayOfWeek
Friday       37.922222
Monday       43.197436
Thursday     41.177273
Tuesday      42.520833
Wednesday    43.170213
Name: TotalTime, dtype: float64

## Fai un delta (diferencia) entre o TotalTime e o MovingTime. Indica cal foi o día con maior diferencia e o de menor diferencia

In [136]:
df_r = pd.DataFrame({"Delta": pd.to_timedelta(pd.to_timedelta(df["TotalTime"], unit="m") - pd.to_timedelta(df["MovingTime"], unit="m")), "DayOfWeek": df["DayOfWeek"]})
df_r.agg(["min","max"])[["Delta","DayOfWeek"]]

,Delta,DayOfWeek
min,0 days 00:00:00,Friday
max,0 days 00:25:48,Wednesday


## Agrupa os datos por día da semana collendo a media de tempo (o delta do apartado anterior) e indica o día da semana de menor e maior diferencia

In [137]:
df_r.groupby("DayOfWeek").mean().reset_index().agg(["min","max"])

,DayOfWeek,Delta
min,Friday,0 days 00:02:48.444444444
max,Wednesday,0 days 00:05:03.076923076


## Elimina as columnas na que máis da metade dos valores sexan nulos

In [138]:
df.isna().sum()

Date                0
StartTime           0
DayOfWeek           0
GoingTo             0
Distance            0
MaxSpeed            0
AvgSpeed            0
AvgMovingSpeed      0
FuelEconomy        17
TotalTime           0
MovingTime          0
Take407All          0
Comments          181
EndTime             0
dtype: int64

In [139]:
df.dropna(thresh = len(df)/2, axis = 1, inplace = True)

## Visualiza o número de nulos de cada columna

In [140]:
df.isna().sum()

Date               0
StartTime          0
DayOfWeek          0
GoingTo            0
Distance           0
MaxSpeed           0
AvgSpeed           0
AvgMovingSpeed     0
FuelEconomy       17
TotalTime          0
MovingTime         0
Take407All         0
EndTime            0
dtype: int64

## Substitúe os valores nules empregando o valor da fila anterior (method='ffil')

In [141]:
df.fillna(method="ffill", inplace=True)

In [142]:
df.isna().sum()

Date              0
StartTime         0
DayOfWeek         0
GoingTo           0
Distance          0
MaxSpeed          0
AvgSpeed          0
AvgMovingSpeed    0
FuelEconomy       6
TotalTime         0
MovingTime        0
Take407All        0
EndTime           0
dtype: int64

## Os primeiros valores son NaN, debemos corrixilo substituíndo estes valores polos anteriores (method='bfill')

In [143]:
df.fillna(method="bfill", inplace=True)


In [144]:
df.isna().sum()

Date              0
StartTime         0
DayOfWeek         0
GoingTo           0
Distance          0
MaxSpeed          0
AvgSpeed          0
AvgMovingSpeed    0
FuelEconomy       0
TotalTime         0
MovingTime        0
Take407All        0
EndTime           0
dtype: int64